In [40]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline

from config import mapbox_api_key, b_api_key

import pandas as pd
import numpy as np

import requests

from pprint import pprint
from datetime import datetime as dt
from datetime import timedelta

from pytz import timezone
import pytz

# Functions to retreive data:

In [2]:
def num_hours_ago(hours):
    """
    Returns UTC time x hours ago.
    """
#     from datetime import datetime as dt
    date = dt.utcnow() - timedelta(hours=hours)
    return dt.strftime(date, "%Y-%m-%dT%H:%M:%S")

In [3]:
def get_coords(location):
    """
    Returns coordinates of address for given input.
    """
    from config import g_api_key
    base_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={g_api_key}"
    
    data = requests.get(base_url).json()
    
    try:
        loc = data['results'][0]['geometry']['location']
        
        lat = loc['lat']
        lng = loc['lng']
    except:
        return "404"
    
    return lat,lng

In [4]:
def pollution(lat,lng):
    """
    Returns Breezometer pollution data from coordinates.
    """
    
    end_datetime = num_hours_ago(1)
    start_datetime = num_hours_ago(168)
    
    try:
        base_url = f"https://api.breezometer.com/air-quality/v2/historical/hourly?lat={lat}&lon={lng}&key={b_api_key}&start_datetime={start_datetime}&end_datetime={end_datetime}"
        data = requests.get(base_url).json()
        return data
    except:
        return None

In [5]:
def pollution_epa(lat,lng):
    """
    Returns EPA pollution data from coordinates.
    """
    
    base_url = f"https://api.breezometer.com/air-quality/v2/current-conditions?lat={lat}&lon={lng}&key={b_api_key}&features=local_aqi"
    data = requests.get(base_url).json()
    return data

In [42]:
def utc_to_pst(date):
    d = pytz.utc.localize(date)
    pst = d.astimezone(timezone('US/Pacific'))
    parsed_pst = dt.strftime(pst, '%Y-%m-%d %I:%M %p')
    
    return parsed_pst

# Mapping functions:

In [51]:
def check_col(df,col):
    
    '''Checks column and returns appropriate parameters for mapping.'''
    
    size = df[col]
    colorscale = 'Jet'
    reversescale=False
    
    if col == 'rain':
        string = "{}<br>Value: {}mm<br>Date: {}"
        size = df[col]*2
        colorscale = 'Blues'
        reversescale=True
        
    elif col == 'wind_speed':
        string = "{}<br>Value: {}m/s<br>Date: {}"
        
    elif col == 'temperature':
        string = "{}<br>Value: {}\u00b0C<br>Date: {}"
        size = df[col]/2
        
    elif col == 'cloud':
        string = "{}<br>Percentage: {}%<br>Date: {}"
        size = df[col]/1.5
        colorscale = 'Blues'
        
    elif col == 'pressure':
        string = "{}<br>Value: {}hPa<br>Date: {}"
        size = df[col]/100
        colorscale='YlGnBu'
        
    elif col == 'aqi':
        string = "{}<br>Value: {} mW/m\u00b2<br>{}<br>Dominant pollutant: {}<br>Date: {}"
        size = df[col]/10
        
    else:
        string = "{}<br>Value: {}<br>Date: {}"
        size = df[col]*2
    
    return size,string,colorscale,reversescale

In [43]:
def marker_text(df,col,fstring):
    '''
    Generates hover-over text for weather info.
    '''
    text = []
    for row in df.itertuples():
        if col == 'aqi':
            a = getattr(row, "city")
            b = getattr(row, col)
            c = getattr(row, "category")
            d = getattr(row, "dominant_pollutant")
            e = utc_to_pst(getattr(row, "date"))
            marker_text = fstring.format(a,b,c,d,e)
            text.append(marker_text)
        else:
            a = getattr(row, "city")
            b = getattr(row, col)
            c = utc_to_pst(getattr(row, "date"))
            marker_text = fstring.format(a,b,c)
            text.append(marker_text)
    return text

In [52]:
def make_traces(df,columns):

    '''
    Generates traces from weather data.
    '''
    data = []
    
    for column in columns:
        
        # Return appropriate data and layout for each column.
        size,string,colorscale,reversescale = check_col(df,column)
        
        # Display first trace instead of all data at once.
        if column == columns[0]:
            visible = True
        else:
            visible = 'legendonly'
            
        trace = go.Scattermapbox(
            lat=round(df['lat'],3),
            lon=round(df['lng'],3),
            mode='markers',
            marker=dict(
                size=size,
                color= size,
                colorscale = colorscale,
                reversescale=reversescale
            ),
            text= marker_text(df,column,string),
            name=column,
            visible=visible
            )
        data.append(trace)
        
    return data

In [10]:
def make_conditions(col_list):
    
    ''' Returns an array with lists of conditions for dropdown menus.'''
    
    # Make array full of False values with dimensions of input list.
    array = np.full(shape=(len(col_list),len(col_list)), fill_value=False, dtype=bool)
    
    for i in range(len(array)):
        array[i][i] = True
    
    return array

In [11]:
def menu_buttons(labels,conditions):
    
    '''Returns a list of buttons for dropdown menus in scattermap.'''
    
    buttons = []
    
    for label,condition in zip(labels,conditions):
        dic = {'label':label,
            'method':'update',
            'args':[{'visible': condition}]}
        
        buttons.append(dic)
        
    return buttons

### Testing plot functions:

In [48]:
df = pd.read_csv('test_data.csv')
df.head()

,city,type,county,state,lat,lng,uv_index,aqi,category,dominant_pollutant,date,temperature,cloud,pressure,wind_speed,rain
0,Adelanto,City,San Bernardino,california,34.582770,-117.409215,2.40,33,Good air quality,pm25,12/4/2018 23:58,57.67,40,1017.0,7.7,0.0
1,Agoura Hills,City,Los Angeles,california,34.153340,-118.761676,2.44,47,Good air quality,pm25,12/4/2018 23:57,66.49,40,1014.0,5.7,0.0
2,Alameda,City,Alameda,california,37.765206,-122.241636,1.64,63,Moderate air quality,pm25,12/5/2018 0:15,52.68,90,1014.0,5.7,0.0
3,Albany,City,Alameda,california,37.886870,-122.297748,1.62,69,Moderate air quality,pm25,12/5/2018 0:15,52.65,75,1014.0,2.1,0.0
4,Alhambra,City,Los Angeles,california,34.095287,-118.127015,2.45,59,Moderate air quality,pm25,12/4/2018 23:58,67.24,1,1014.0,4.6,0.0


In [13]:
# for column in df.columns:
#     print(f"{column}:  {check_col(column)}")
# for row in df.itertuples():
#     print(row.lat)
#     break

In [49]:
def generate_scattermap(df):
    
    # lists for menu and for actual plotting.
    menu_categories = ['UV Index', 'AQI', 'Temperature', 'Cloud', 'Pressure', 'Wind Speed', 'Rain']
    plot_columns = ['uv_index', 'aqi','temperature', 'cloud','pressure', 'wind_speed', 'rain']

    # menu options and data for plotting
    conditions = make_conditions(menu_categories)
    buttons = menu_buttons(menu_categories,conditions)

    data = make_traces(df,plot_columns)

    updatemenus = list([
        dict(
             buttons=buttons,
            x = 0.0,
            xanchor = 'left',
            y = 1,
            yanchor = 'top',
            pad = {'l': 3, 't': 3},
            bgcolor = '#AAAAAA',
            showactive = True,
            bordercolor = '#FFFFFF',
            font = dict(size=11, color='#000000')
        )])

    layout = go.Layout(
        autosize=True,
        hovermode='closest',
        mapbox=dict(
            accesstoken=mapbox_api_key,
            bearing=0,
            center=dict(
                lat=36,
                lon=-119
            ),
            style='dark',
            pitch=0,
            zoom=4
        ),
        margin = dict( t=0, b=0, l=0, r=0 ),
        updatemenus=updatemenus,
        showlegend=False
    )
    
    fig = dict(data=data, layout=layout)
    map_html = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')
    plot = py.iplot(fig, filename='test_map')
    return plot

In [54]:
generate_scattermap(df)

AttributeError: 'str' object has no attribute 'tzinfo'

# Appendix:

In [16]:
# x,y = get_coords("beijing")

In [17]:
# print(x)
# pollution_data = pollution(x,y)

In [6]:
# def generate_map(df,size,color):
#     '''
#     Generates HTML for map from pollution data.
#     '''
#     data = [
#     go.Scattermapbox(
#         lat=round(df['lat'],3),
#         lon=round(df['lng'],3),
#         mode='markers',
#         marker=dict(
#             size=df[size],
#             color= df[color],
#             colorscale = 'Blues',
#             reversescale=True
#         ),
# #         text= marker_text(df)
#         )
#     ]

#     layout = go.Layout(
#         autosize=True,
#         hovermode='closest',
#         mapbox=dict(
#             accesstoken=mapbox_api_key,
#             bearing=0,
#             center=dict(
#                 lat=36,
#                 lon=-119
#             ),
#             style='dark',
#             pitch=0,
#             zoom=4
#         ),
#         margin=dict(
#         l= 0,
#         r= 0,
#         b= 0,
#         t= 0,
#         pad= 2
#         )
#     )

#     fig = dict(data=data, layout=layout)

# #     map_html = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')
#     plot = py.iplot(fig, filename='test_map')
    
#     return plot

In [18]:
# def df_geodata(filename=None,query=None):    
#     """
#     Returns dataframe with pollution data from city list dataframe. Makes API calls via 'pollution_epa'
#     to scrape latest data.
#     """
    
#     if filename and query:
#         return "You must pass either a csv filename or a SQL query. You cannot pass two arguments."
#     elif filename == None and query == None:
#         return "Error: No arguments were passed to this function."
#     elif filename:
#         df = pd.read_csv(filename)
#     else:
#         df = pd.read_sql_query(query).set_index('index')
    
#     aqi = []
#     category = []
#     dominant_pollutant = []
#     date = []


#     for lat,lng in zip(df['lat'],df['lng']):
#         # Get pollution data.
#         data = pollution_epa(lat,lng)
        
#         if data != None:
#             index = data['data']['indexes']['usa_epa']

#             air_quality = index['aqi']
#             categories = index['category']
#             dom_pollutant = index['dominant_pollutant']
#             datetime = data['data']['datetime']

#             aqi.append(air_quality)
#             category.append(categories)
#             dominant_pollutant.append(dom_pollutant)
#             date.append(datetime)
            
#         else:
#             aqi.append("NaN")
#             category.append("NaN")
#             dominant_pollutant.append("NaN")
#             date.append("NaN")          
    
#     # Update columns with latest data
#     df['aqi'] = aqi
#     df['category'] = category
#     df['dominant_pollutant'] = dominant_pollutant
#     df['datetime'] = date
    
#     return df

In [19]:
# generate_plot("san francisco")

# x = [x['datetime'] for x in pollution_data['data'] if x['data_available'] == True]
# y = [y['indexes']['baqi']['aqi'] for y in pollution_data['data'] if y['data_available'] == True]
# text1 = [text['indexes']['baqi']['dominant_pollutant'] for text in pollution_data['data'] if text['data_available'] == True]
# text2 = [text['indexes']['baqi']['category'] for text in pollution_data['data'] if text['data_available'] == True]

# test = [f"Dominant pollutant: {x}\n{y}" for x,y in zip(text1,text2)]

# data = [go.Scattergl(x=x,
#                    y=y, 
#                    mode="markers", 
#                    marker=dict(
#                     size=10,
#                     color = y, #set color equal to a variable
#                     colorscale='Viridis',
#                     showscale=True,
#                     reversescale=True
#                    ),  
#                    name="AQI",
#                     text= test)]

# layout = go.Layout(title="AQI for beijing",
#                 xaxis=dict(title='Date'),
#                 yaxis=dict(title='AQI'))

# fig = go.Figure(data=data, layout=layout)

# # py.iplot(fig, filename='jupyter-basic_line')

# p = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')

In [20]:
# def marker_text(df):
#     '''
#     Generates hover-over text for AQI info.
#     '''
#     text = []
#     for row in df.itertuples():
#         a = getattr(row, "city")
#         b = getattr(row, "category")
#         c = getattr(row, "aqi")
#         d = getattr(row, "dominant_pollutant")
#         string = f"{a}<br>{b}<br>AQI: {c}<br>Dominant pollutant: {d}"
#         text.append(string)
#     return text

In [21]:
# def make_traces(size,colorscale,marker_text,name,reversescale=False,visible='legendonly'):

#     '''
#     Generates traces from pollution data.
#     '''
#     trace = go.Scattermapbox(
#         lat=round(df['lat'],3),
#         lon=round(df['lng'],3),
#         mode='markers',
#         marker=dict(
#             size=size,
#             color= size,
#             colorscale = colorscale,
#             reversescale=reversescale
#         ),
#         text= marker_text,
#         name=name,
#         visible=visible
#         )
    
#     return trace

In [22]:
# print(p)
# print(text1)
# print(text2)
# print(test)
# x,y = "404"
# print(p)

In [23]:

# df_wind = pd.read_csv('https://plot.ly/~datasets/2805.csv')

# df_known_capacity = df_wind[ df_wind['total_cpcy'] != -99999.000 ]
# df_sum = df_known_capacity.groupby('manufac')['total_cpcy'].sum().sort_values(ascending=False).to_frame()

# df_farms = pd.read_csv('https://plot.ly/~jackp/17256.csv')
# df_farms.set_index('Wind Farm', inplace=True)

# wind_farms=list([
#     dict(
#         args=[ { 
#             'mapbox.center.lat':38,
#             'mapbox.center.lon':-94,
#             'mapbox.zoom':3,
#             'annotations[0].text':'All US wind turbines (scroll to zoom)'
#         } ],
#         label='USA',
#         method='relayout'
#     )
# ])

# for farm, row in df_farms.iterrows():
#     desc = []
#     for col in df_farms.columns:
#         if col not in ['DegMinSec','Latitude','Longitude']:
#             if str(row[col]) not in ['None','nan','']: 
#                 desc.append( col + ': ' + str(row[col]).strip("'") )
#     desc.insert(0, farm)
#     wind_farms.append( 
#         dict(
#             args=[ { 
#                 'mapbox.center.lat':row['Latitude'], 
#                 'mapbox.center.lon':float(str(row['Longitude']).strip("'")), 
#                 'mapbox.zoom':9,
#                 'annotations[0].text': '<br>'.join(desc)
#             } ],
#             label=' '.join(farm.split(' ')[0:2]),
#             method='relayout'
#         )
#     )

# data = []
# for mfr in list(df_sum.index):
#     if mfr != 'unknown':
#         trace = dict(
#             lat = df_wind[ df_wind['manufac'] == mfr ]['lat_DD'],
#             lon = df_wind[ df_wind['manufac'] == mfr ]['long_DD'],
#             name = mfr,
#             marker = dict(size = 4),
#             type = 'scattermapbox'
#         )
#     data.append(trace)

# # mapbox_access_token = 'insert mapbox token here'

# layout = dict(
#     height = 800,
#     margin = dict( t=0, b=0, l=0, r=0 ),
#     font = dict( color='#FFFFFF', size=11 ),
#     paper_bgcolor = '#000000',
#     mapbox=dict(
#         accesstoken=mapbox_api_key,
#         bearing=0,
#         center=dict(
#             lat=38,
#             lon=-94
#         ),
#         pitch=0,
#         zoom=3,
#         style='dark'
#     ),
# )

# updatemenus=list([
#     dict(
#         buttons = wind_farms[0:10],
#         pad = {'r': 0, 't': 10},
#         x = 0.1,
#         xanchor = 'left',
#         y = 1.0,
#         yanchor = 'top',
#         bgcolor = '#AAAAAA',
#         active = 99,
#         bordercolor = '#FFFFFF',
#         font = dict(size=11, color='#000000')
#     ),
#     dict(
#         buttons=list([
#             dict(
#                 args=['mapbox.style', 'dark'],
#                 label='Dark',
#                 method='relayout'
#             ),                    
#             dict(
#                 args=['mapbox.style', 'light'],
#                 label='Light',
#                 method='relayout'
#             ),
#             dict(
#                 args=['mapbox.style', 'satellite'],
#                 label='Satellite',
#                 method='relayout'
#             ),
#             dict(
#                 args=['mapbox.style', 'satellite-streets'],
#                 label='Satellite with Streets',
#                 method='relayout'
#             )                    
#         ]),
#         direction = 'up',
#         x = 0.75,
#         xanchor = 'left',
#         y = 0.05,
#         yanchor = 'bottom',
#         bgcolor = '#000000',
#         bordercolor = '#FFFFFF',
#         font = dict(size=11)
#     ),        
# ])

# annotations = list([
#     dict(text='All US wind turbines (scroll to zoom)', font=dict(color='magenta',size=14), borderpad=10, 
#          x=0.05, y=0.05, xref='paper', yref='paper', align='left', showarrow=False, bgcolor='black'),
#     dict(text='Wind<br>Farms', x=0.01, y=0.99, yref='paper', align='left', showarrow=False,font=dict(size=14))
# ])

# layout['updatemenus'] = updatemenus
# layout['annotations'] = annotations

# figure = dict(data=data, layout=layout)
# py.iplot(figure, filename='wind-turbine-territory-dropdown')

In [24]:
# data = [
#     go.Scattermapbox(
#         lat=round(df['lat'],3),
#         lon=round(df['lng'],3),
#         mode='markers',
#         marker=dict(
#             size=df['aqi']/10,
#             color= df['aqi'],
#             colorscale = 'Jet',
#         ),
#         text= marker_text(df)
#     )
# ]

# layout = go.Layout(
#     autosize=True,
#     hovermode='closest',
#     mapbox=dict(
#         accesstoken=mapbox_api_key,
#         bearing=0,
#         center=dict(
#             lat=36,
#             lon=-119
#         ),
#         style='dark',
#         pitch=4,
#         zoom=4
#     ),

#     margin=dict(
#     l= 0,
#     r= 0,
#     b= 0,
#     t= 0,
#     pad= 2
#     )
# )

# fig = dict(data=data, layout=layout)

# py.iplot(fig, filename='CA Mapbox')
# py.iplot(fig, filename='CA Mapbox')
# cali_plot = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')

In [25]:
# print(cali_plot)
# cond1=[True,False,False,False,False,False,False]
# cond2=[False,True,False,False,False,False,False]
# cond3=[False,False,True,False,False,False,False]
# cond4=[False,False,False,True,False,False,False]
# cond5= cond3[::-1]
# cond6= cond2[::-1]
# cond7= cond1[::-1]

### Scatterplot function:

In [26]:
# def generate_plot(city):  
#     '''
#     Generates plot HTML for AQI for past week.
#     '''
#     try:    
#         lat,lng = get_coords(city)
#     except ValueError:
#         return "Error: We could not find the coordinates for the entry you supplied."

#     pollution_data = pollution(lat,lng)
    
#     if pollution_data != None:
#         try:
#             x = [x['datetime'] for x in pollution_data['data'] if x['data_available'] == True]
#             y = [y['indexes']['baqi']['aqi'] for y in pollution_data['data'] if y['data_available'] == True]

#             text1 = [text['indexes']['baqi']['dominant_pollutant'] for text in pollution_data['data'] if text['data_available'] == True]
#             text2 = [text['indexes']['baqi']['category'] for text in pollution_data['data'] if text['data_available'] == True]

#             test = [f"Dominant pollutant: {x}\n{y}" for x,y in zip(text1,text2)]

#             data = [go.Scattergl(x=x,
#                                y=y, 
#                                mode="markers", 
#                                marker=dict(
#                                 size=12,
#                                 color = y, #set color equal to a variable
#                                 colorscale='Viridis',
#                                 showscale=True
#                                ),  
#                                name="AQI",
#                                 text= test)]

#             layout = go.Layout(title=f"AQI for {city.title()}",
#                             xaxis=dict(title='Date'),
#                             yaxis=dict(title='AQI'))

#             fig = go.Figure(data=data, layout=layout)
            
#             return py.plot(fig, filename='jupyter-basic_line',auto_open=False)
        
#         except:
#             return "404"        